In [257]:
# Chase Abram
# using Pkg
# Pkg.add("DataFrames")
# using CSV
# using DataFrames
# # df = DataFrame()
# df = CSV.read("psetOne.csv", DataFrame)

using LinearAlgebra

using Random
rng = MersenneTwister(12345)

MersenneTwister(UInt32[0x00003039], Random.DSFMT.DSFMT_state(Int32[-870096391, 1072918504, -1812426662, 1073255081, -733866021, 1073404543, 807620846, 1073368448, 1919433844, 1072852359  …  -362113007, 1073100625, -166402106, 1073460158, -1907020342, 721295190, -750225566, -1300227565, 382, 0]), [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], UInt128[0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000  …  0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x000000000

In [161]:
names(df)
df17 = df[df."Market" .== 17, :]
# df17."shares"
# X = convert(Matrix, df17[:, filter(x -> x in ["Constant", "EngineSize", "SportsBike", "Brand2", "Brand3"], names(df17))])
Br = convert(Matrix, df17[:, filter(x -> x in ["Brand2", "Brand3"], names(df17))])
Br = Bool.(Br)

7×2 BitArray{2}:
 0  0
 0  0
 1  0
 1  0
 0  1
 0  1
 0  1

In [324]:
function u_bar_jt(delta, sigma, X, zeta)
    return delta .+ X*sigma*zeta
end

function Prij(u_bar)
    return exp.(u_bar)./sum(exp.(u_bar))
end


function s_logit(p, alpha, beta, X, xi)
    # Computer nmuerators for each s_j
    nums = exp.(X*beta .- alpha.*p[2:end] .+ xi)
    
    # return s_j, starting with s_0
    return [1/(1 + sum(nums)); nums./(1 + sum(nums))]
end

function s_logit_jac(p, alpha, beta, X, xi)
    # Get shares
    sp = s_logit(p, alpha, beta, X, xi)
    
    # Jacobian is the outer product times alpha in this case...
    jac = alpha .* sp * sp'
    
    # ... except for an extra term on the diag
    for j in 1: length(jac[:,1])
        jac[j,j] -= alpha*sp[j]
    end
    
    return jac
end

function s_logit_hess(p, alpha, beta, X, xi)
    hess = zeros(length(p), length(p), length(p))
    for j in 1:length(p)
        for k in 1:length(p)
            for l in 1:length(p)
                if j == k && j == l
                    hess[j,k,l] = s_logit_jac(p, alpha, beta, X, xi)[j,j]*alpha*(2*s_logit(p, alpha, beta, X, xi)[j] - 1)
                elseif j == k
                    hess[j,k,l] = s_logit_jac(p, alpha, beta, X, xi)[l,j]*alpha*(2*s_logit(p, alpha, beta, X, xi)[j] - 1)
                elseif j == l
                    hess[j,k,l] = s_logit_jac(p, alpha, beta, X, xi)[k,j]*alpha*(2*s_logit(p, alpha, beta, X, xi)[j] - 1)
                elseif k == l
                    hess[j,k,l] = alpha*(s_logit_jac(p, alpha, beta, X, xi)[k,j] + s_logit_jac(p, alpha, beta, X, xi)[l,j])
                else
                    hess[j,k,l] = alpha*(s_logit_jac(p, alpha, beta, X, xi)[k,j] + s_logit_jac(p, alpha, beta, X, xi)[l,k])
                end
            end
        end
    end
    
    return hess
end

function Omega_star(br)
    Om = zeros(size(br,1) + 1,size(br,1) + 1)
    for i in 1:size(Om,1) - 1
        for j in 1:size(Om,2) - 1
            if br[i,1] && br[j,1]
                Om[i+1,j+1] = 1
            elseif br[i,2] && br[j,2]
                Om[i+1,j+1] = 1
            elseif (!br[i,1]&& !br[j,1]) && (!br[i,2]&& !br[j,2])
                Om[i+1,j+1] = 1
            end
        end
    end
    Om[1,1] = 1
    return Om
end

function fp_logit(p, alpha, beta, X, xi, br)
    return s_logit(p, alpha, beta, X, xi) - Omega_star(br).* s_logit_jac(p, alpha, beta, X, xi)*p
end

# function fp_logit_jac(p, alpha, beta, X, xi, br)
#     first_term = s_logit_jac(p, alpha, beta, X, xi)
#     second_term = Omega_star(br) .* s_logit_jac(p, alpha, beta, X, xi)
#     third_term = zeros(length(p), length(p))
    
#     hess = s_logit_hess(p, alpha, beta, X, xi)
    
#     for i in 1:length(p)
#         for j in 1:length(p)
# #             println("size: ", size((p'*(hess[i,j,:].* Omega_star(br)[i,j]))))
#             third_term[i,j] = (p'*(hess[i,j,:].* Omega_star(br)[i,j]))[1]
#         end
#     end
    
#     return first_term + second_term + third_term
    
# end

function jac_om_p(p, alpha, beta, X, xi, br)
    jacomp = zeros(length(p), length(p))
    hess = s_logit_hess(p, alpha, beta, X, xi)
    for i in 1:length(p)
        for j in 1:length(p)
            jacomp[i,j] = (p'*(hess[i,:,j].* Omega_star(br)[i,j]))[1]
        end
    end
    return jacomp
end


function fp_logit_jac(p, alpha, beta, X, xi, br)
    sp = s_logit(p, alpha, beta, X, xi)
    Om = Omega_star(br).* s_logit_jac(p, alpha, beta, X, xi)
    s_jac = s_logit_jac(p, alpha, beta, X, xi)
    jacomp = jac_om_p(p, alpha, beta, X, xi, br)
    
    return ((sp - Om*p)' * (s_jac - jacomp - Om))'
end





function solve_fp_logit(alpha, beta, X, xi, br, tol = 1e-14, maxit = 100)
    p = ones(size(X,1) + 1)
    it = 0
    dif = Inf
    while it < maxit && dif > tol
        J = fp_logit_jac(p, alpha, beta, X, xi, br)
        p += J
#         p /= sum(p)
        dif = maximum(abs.(fp_logit(p, alpha, beta, X, xi)))
        it += 1
        println("it: ", it, ", dif: ", dif)
#         println("fp_logit: ", fp_logit(p, alpha, beta, X, xi))
#         println("s: ", s_logit(p, alpha, beta, X, xi))
    end
    
    return p
end



solve_fp_logit (generic function with 3 methods)

In [325]:
p = ones(size(X,1)+1,1)
p[7] = 1
alpha = 3
beta = [1 1 2 -1 1]'

rng = MersenneTwister(1234)
xi = randn(rng, size(df17,1))

s_logit(p, alpha, beta, X, xi)
println("sum shares: ", sum(s_logit(p, alpha, beta, X, xi)))

s_logit_jac(p, alpha, beta, X, xi)

s_logit_hess(p, alpha, beta, X, xi)

Omega_star(Br)

# Omega(Br)

fp_logit(p, alpha, beta, X, xi, Br)



jac_om_p(p, alpha, beta, X, xi, Br)

fp_logit_jac(p, alpha, beta, X, xi, Br)

solve_fp_logit(alpha, beta, X, xi, Br)

# df17[3, "Brand2"]

sum shares: 1.0
it: 1, dif: 1.085189159226589
it: 2, dif: 0.959963528373179
it: 3, dif: 0.9629011542644781
it: 4, dif: 0.9661932984809263
it: 5, dif: 0.9695115541368959
it: 6, dif: 0.9726377944680767
it: 7, dif: 0.9754616575177354
it: 8, dif: 0.9779497437739225
it: 9, dif: 0.98011294550683
it: 10, dif: 0.9819827547323103
it: 11, dif: 0.9835972110230548
it: 12, dif: 0.9849937288508517
it: 13, dif: 0.9862060045054928
it: 14, dif: 0.9872630706543355
it: 15, dif: 0.9881893669837376
it: 16, dif: 0.9890052254626163
it: 17, dif: 0.9897274740581635
it: 18, dif: 0.9903700251399192
it: 19, dif: 0.9909443963686602
it: 20, dif: 0.9914601505469012
it: 21, dif: 0.9919252578336967
it: 22, dif: 0.9923463900517068
it: 23, dif: 0.992729158227877
it: 24, dif: 0.9930783038281519
it: 25, dif: 0.9933978527206337
it: 26, dif: 0.9936912393452475
it: 27, dif: 0.9939614071368895
it: 28, dif: 0.9942108900285325
it: 29, dif: 0.9944418788599645
it: 30, dif: 0.9946562757152893
it: 31, dif: 0.9948557385763703
it: 32

8×1 Array{Float64,2}:
  1.0000103491168375
  1.0004933689867601
  1.0006220198149547
  1.000046523678116
  1.000228987142522
 -0.18729607997970493
  1.3302886134203538
 -2.99836317867114

In [ ]:
function sHat(delta, X, sigma, zeta)
    
    
    # Compute u_bar_ijt
    
    # Compute prob.
    
    # Do above I times, then integrate
    
    
    return 0
end

